In [1]:
#Check Python Version
import shutil
import sys
import os
sys.version


#Upgrade pip
%pip install --upgrade pip


#Check TF & TFX Versioning
import tensorflow as tf
print(tf.__version__)
from tfx import v1 as tfx
print(tfx.__version__)


# Pipeline name
PIPELINE_NAME = "copy_tfrecords"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = './artifacts'
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Folder path to data
DATA_ROOT = './data/'

# Folder path to tfrecords
# TFRECORDS_TRAIN_DATA_PATH = '../../../../Documents/tfrecord1_split_train/data_tfrecord-00000-of-00001.gz'
# TFRECORDS_EVAL_DATA_PATH = '../../../../Documents/tfrecord2_split_eval/data_tfrecord-00000-of-00001.gz'
TFRECORDS_TRAIN_DATA_PATH = '../../../../Documents/tfrecord1_split_train/'
TFRECORDS_EVAL_DATA_PATH = '../../../../Documents/tfrecord2_split_eval/'
from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

Note: you may need to restart the kernel to use updated packages.


2023-03-13 22:01:44.255185: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 22:01:44.433352: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 22:01:44.433415: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-13 22:01:45.751839: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

2.11.0
1.12.0


In [2]:
import json
import tempfile
from typing import Any, Dict, List
from tfx.v1.types.standard_artifacts import Examples
from tfx.types import artifact_utils
from typing import Any, Dict, List, Optional, Type, Union


from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from hello import component

from tfx.dsl.component.experimental.decorators import component

from tfx.dsl.io import fileio

@component
def CopyExampleGen(
        input_dict: tfx.dsl.components.Parameter[Dict[str, str]],
        output_example: tfx.dsl.components.OutputArtifact[Examples]
      ) -> tfx.dsl.components.OutputDict():
    
        """TODO: Function that parses input_dict, creates a directory from the split-names and tfrecord uris provided"""

        input_train_dir='./examples_imported/Split-train/data_tfrecord-00000-of-00001.gz'
        input_eval_dir='examples_imported/Split-eval/data_tfrecord-00000-of-00001.gz'
        split_names='["train","eval"]'

        output_example.split_names=artifact_utils.encode_split_names(['train', 'eval'])

        fileio.mkdir(output_example.uri+'/Split-train/')
        fileio.mkdir(output_example.uri+'/Split-eval/')

        fileio.copy(input_train_dir, output_example.uri+'/Split-train/data_tfrecord-00000-of-00001.gz', True) 
        fileio.copy(input_eval_dir, output_example.uri+'/Split-eval/data_tfrecord-00000-of-00001.gz', True) 


def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  # User input dictionary with split-names and their resepctive uri to tfrecords
  # tfrecords_dict: Dict[str, str]={
  #   "train":TFRECORDS_TRAIN_DATA_PATH,
  #   "eval":TFRECORDS_EVAL_DATA_PATH
  # }
  # split_names=''
  # counter=1

#   cwd = os.getcwd()
#   temp_dir = tempfile.TemporaryDirectory(dir = cwd)

#   print(temp_dir.name)

  # Destination directory for source
  # destination_examples_artifact_uri = './examples'
  # print(destination_examples_artifact_uri)

  # if not os.path.exists(destination_examples_artifact_uri):
  #     os.mkdir(destination_examples_artifact_uri)


  # # Create a folder in destination_examples_artifact_uri for every key name
  # for key in tfrecords_dict:
  #       split_name_temp = "Split-"+key
  #       path = os.path.join(destination_examples_artifact_uri, split_name_temp)
  #       if not os.path.exists(path):
  #           os.mkdir(path)
  #           print(path)
  #       shutil.copy(tfrecords_dict[key], path)
  #       print(path)


  # # Create split_names string
  # for key in tfrecords_dict:
  #     if(counter==1):
  #         split_names+='["'+key+'","'
  #         counter=counter+1
  #     elif(counter==2):
  #         split_names+=key+'"]'
  #     else:
  #         split_names+=key+','
  #         counter=counter+1

  # print(split_names)
  # example_gen=tfx.components.CsvExampleGen(input_base='./data/data.csv')

  # # CopyExampleGen execution process to replace Importer Node functionality
  # examples_importer = tfx.dsl.Importer(
  #       source_uri=destination_examples_artifact_uri,
  #       artifact_type=tfx.types.standard_artifacts.Examples,
  #       # properties={"split_names": '["train","eval"]'}
  #       properties={"split_names": split_names}
  #       ).with_id('examples_importer')


#   output_example: tfx.dsl.components.OutputArtifact[tfx.types.standard_artifacts.Examples]
                                                    
  # copy_examples=CopyExampleGen(
  #     input_dict=tfrecords_dict,
  #   #   output_example=output_example
  # )

  
#   hello_component = component.HelloComponent(
#       input_data=examples_importer.outputs['result'],
#       tfrecords_dict=tfrecords_dict
#     #   input_data=tfrecords_dict
#   )
  example_gen=tfx.components.CsvExampleGen(input_base='./data/')

  # Test downstream component
  # statistics_gen = tfx.components.StatisticsGen(
  #    examples=example_gen.outputs['example'])

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      # copy_examples,
      # examples_importer,
    #   hello_component,
      # statistics_gen
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

In [3]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=METADATA_PATH)
  )

INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
custom_driver_specs {
  key: "CsvExampleGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.example_gen.driver.FileBasedDriver"
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "metadata/copy_tfrecords/metadata.db"
      connection_mode: READWRITE_OPENCREATE
    }
  }
}

INFO:absl:Using connection config:
 sqlite {
  filename_uri: "metadata/copy_tfrecords/metadata.db"
  connection_mode: READWRITE_OPENCREATE
}

INFO:absl:Component CsvExampleGen is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.example_gen.csv_example_gen.component.CsvExampleGen"
  }
  id: "CsvExampleGen"
}
contexts {
  contexts {
    type {
      name: "

INFO:absl:Processing input csv data ./data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Value type <class 'NoneType'> of key version in exec_properties is not supported, going to drop it
INFO:absl:Value type <class 'list'> of key _beam_pipeline_args in exec_properties is not supported, going to drop it
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 6 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "./artifacts/CsvExampleGen/examples/6"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:13137114,xor_checksum:1678744741,sum_checksum:1678744741"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
, artifact_type: name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  valu

In [ ]:
import re
'./examples/'
dict_output = {
	"name": "NotFoundError",
	"message": "",
	"stack": "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m\n\u001b[0;31mNotFoundError\u001b[0m                             Traceback (most recent call last)\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/dsl/io/plugins/tensorflow_gfile.py\u001b[0m in \u001b[0;36mmkdir\u001b[0;34m(path)\u001b[0m\n\u001b[1;32m     75\u001b[0m       \u001b[0;32mtry\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 76\u001b[0;31m         \u001b[0mtf\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mio\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mgfile\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mmkdir\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mpath\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     77\u001b[0m       \u001b[0;32mexcept\u001b[0m \u001b[0mtf\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0merrors\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mNotFoundError\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0me\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tensorflow/python/lib/io/file_io.py\u001b[0m in \u001b[0;36mcreate_dir_v2\u001b[0;34m(path)\u001b[0m\n\u001b[1;32m    482\u001b[0m   \"\"\"\n\u001b[0;32m--> 483\u001b[0;31m   \u001b[0m_pywrap_file_io\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mCreateDir\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mcompat\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mpath_to_bytes\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mpath\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    484\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;31mNotFoundError\u001b[0m: /tmp/tfx-interactive-2023-03-13T18_58_59.825612-ot2c9dut/CopyExampleGen/output_example/1/Split-eval/); No such file or directory\n\nThe above exception was the direct cause of the following exception:\n\n\u001b[0;31mNotFoundError\u001b[0m                             Traceback (most recent call last)\n\u001b[0;32m/tmp/ipykernel_2140447/2256786037.py\u001b[0m in \u001b[0;36m<cell line: 15>\u001b[0;34m()\u001b[0m\n\u001b[1;32m     13\u001b[0m \u001b[0mcopy_examples\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mCopyExampleGen\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0minput_dict\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0mtfrecords_dict\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     14\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 15\u001b[0;31m \u001b[0mcontext\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrun\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mcopy_examples\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     16\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     17\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/experimental/interactive/notebook_utils.py\u001b[0m in \u001b[0;36mrun_if_ipython\u001b[0;34m(*args, **kwargs)\u001b[0m\n\u001b[1;32m     29\u001b[0m       \u001b[0;31m# __IPYTHON__ variable is set by IPython, see\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     30\u001b[0m       \u001b[0;31m# https://ipython.org/ipython-doc/rel-0.10.2/html/interactive/reference.html#embedding-ipython.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 31\u001b[0;31m       \u001b[0;32mreturn\u001b[0m \u001b[0mfn\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     32\u001b[0m     \u001b[0;32melse\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     33\u001b[0m       logging.warning(\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/experimental/interactive/interactive_context.py\u001b[0m in \u001b[0;36mrun\u001b[0;34m(self, component, enable_cache, beam_pipeline_args)\u001b[0m\n\u001b[1;32m    162\u001b[0m         \u001b[0mtelemetry_utils\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mLABEL_TFX_RUNNER\u001b[0m\u001b[0;34m:\u001b[0m \u001b[0mrunner_label\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    163\u001b[0m     }):\n\u001b[0;32m--> 164\u001b[0;31m       \u001b[0mexecution_id\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mlauncher\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mlaunch\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mexecution_id\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    165\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    166\u001b[0m     return execution_result.ExecutionResult(\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/launcher/base_component_launcher.py\u001b[0m in \u001b[0;36mlaunch\u001b[0;34m(self)\u001b[0m\n\u001b[1;32m    204\u001b[0m       \u001b[0;31m# be immutable in this context.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    205\u001b[0m       \u001b[0;31m# output_dict can still be changed, specifically properties.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 206\u001b[0;31m       self._run_executor(execution_decision.execution_id,\n\u001b[0m\u001b[1;32m    207\u001b[0m                          \u001b[0mcopy\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdeepcopy\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mexecution_decision\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0minput_dict\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    208\u001b[0m                          \u001b[0mexecution_decision\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0moutput_dict\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/launcher/in_process_component_launcher.py\u001b[0m in \u001b[0;36m_run_executor\u001b[0;34m(self, execution_id, input_dict, output_dict, exec_properties)\u001b[0m\n\u001b[1;32m     71\u001b[0m     \u001b[0;31m# be immutable in this context.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     72\u001b[0m     \u001b[0;31m# output_dict can still be changed, specifically properties.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 73\u001b[0;31m     executor.Do(\n\u001b[0m\u001b[1;32m     74\u001b[0m         copy.deepcopy(input_dict), output_dict, copy.deepcopy(exec_properties))\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/dsl/component/experimental/decorators.py\u001b[0m in \u001b[0;36mDo\u001b[0;34m(self, input_dict, output_dict, exec_properties)\u001b[0m\n\u001b[1;32m    229\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    230\u001b[0m     \u001b[0;31m# Call function and check returned values.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 231\u001b[0;31m     \u001b[0moutputs\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_FUNCTION\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m**\u001b[0m\u001b[0mfunction_args\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    232\u001b[0m     \u001b[0moutputs\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0moutputs\u001b[0m \u001b[0;32mor\u001b[0m \u001b[0;34m{\u001b[0m\u001b[0;34m}\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    233\u001b[0m     output_dict.update(\n\n\u001b[0;32m/tmp/ipykernel_2140447/1283998052.py\u001b[0m in \u001b[0;36mCopyExampleGen\u001b[0;34m(input_dict, output_example)\u001b[0m\n\u001b[1;32m     25\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     26\u001b[0m         \u001b[0mfileio\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mmkdir\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0moutput_example\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0muri\u001b[0m\u001b[0;34m+\u001b[0m\u001b[0;34m'/Split-train/'\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 27\u001b[0;31m         \u001b[0mfileio\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mmkdir\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0moutput_example\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0muri\u001b[0m\u001b[0;34m+\u001b[0m\u001b[0;34m'/Split-eval/)'\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     28\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     29\u001b[0m         \u001b[0mfileio\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcopy\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0minput_train_dir\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0moutput_example\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0muri\u001b[0m\u001b[0;34m+\u001b[0m\u001b[0;34m'/Split-train/data_tfrecord-00000-of-00001.gz)'\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;32mTrue\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/dsl/io/fileio.py\u001b[0m in \u001b[0;36mmkdir\u001b[0;34m(path)\u001b[0m\n\u001b[1;32m     83\u001b[0m \u001b[0;32mdef\u001b[0m \u001b[0mmkdir\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mpath\u001b[0m\u001b[0;34m:\u001b[0m \u001b[0mPathType\u001b[0m\u001b[0;34m)\u001b[0m \u001b[0;34m->\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     84\u001b[0m   \u001b[0;34m\"\"\"Make a directory at the given path; parent directory must exist.\"\"\"\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 85\u001b[0;31m   \u001b[0m_get_filesystem\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mpath\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mmkdir\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mpath\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     86\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     87\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/dsl/io/plugins/tensorflow_gfile.py\u001b[0m in \u001b[0;36mmkdir\u001b[0;34m(path)\u001b[0m\n\u001b[1;32m     76\u001b[0m         \u001b[0mtf\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mio\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mgfile\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mmkdir\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mpath\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     77\u001b[0m       \u001b[0;32mexcept\u001b[0m \u001b[0mtf\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0merrors\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mNotFoundError\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0me\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 78\u001b[0;31m         \u001b[0;32mraise\u001b[0m \u001b[0mfilesystem\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mNotFoundError\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m \u001b[0;32mfrom\u001b[0m \u001b[0me\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     79\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     80\u001b[0m     \u001b[0;34m@\u001b[0m\u001b[0mstaticmethod\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;31mNotFoundError\u001b[0m: "
}
ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
result = ansi_escape.sub('', dict_output["stack"])

print(result)

In [3]:
from typing import Dict, List
from tfx.v1.types.standard_artifacts import Examples
from tfx.types import artifact_utils
from typing import Dict

from tfx.dsl.component.experimental.decorators import component

from tfx.dsl.io import fileio

def _string_builder(split_names_list: List):
    str1 = "["
    urlist_len = len(split_names_list)-1
    index = 0

    for ele in split_names_list:
        if(index==urlist_len):
            str1 += "\""+ele+"\""+"]"
            break
        str1 += "\""+ele+"\""+","
        index+=1
    return str1

@component
def CopyExampleGen(
        input_dict: tfx.dsl.components.Parameter[Dict[str, str]],
        output_example: tfx.dsl.components.OutputArtifact[Examples]
      ) -> tfx.dsl.components.OutputDict():
    
        """Parse input_dict: creates a directory from the split-names and tfrecord uris provided"""
        split_names=[]
        for key in input_dict:
          split_names.append(key)
          
        split_names_string=_string_builder(split_names)
        output_example.split_names=str(split_names_string)
        
        """Make directories"""
        tfrecords_list=[]
        for key in input_dict:
          split_value="/Split-"+key+"/"
          fileio.mkdir(output_example.uri+split_value)
          tfrecords_list=fileio.glob(input_dict[key]+'*.gz')
          # print("tfrecords_list: " + str(tfrecords_list))

          """"Copy files into directories"""
          for tfrecord in tfrecords_list:
                """TODO: Find a better way to extra file name"""
                file_name=os.path.basename(os.path.normpath(tfrecord))
                fileio.copy(tfrecord, output_example.uri+split_value+file_name, True)
                # print("file_name: "+file_name)

In [4]:
from typing import Dict
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

"""Prepare variables"""
context=InteractiveContext()
tfrecords_dict: Dict[str, str]={
    "train":TFRECORDS_TRAIN_DATA_PATH,
    "eval":TFRECORDS_EVAL_DATA_PATH
  }

# example_gen=tfx.components.CsvExampleGen(input_base='./data/data.csv')

# context.run(example_gen)

"""Call CopyExampleGen Component"""
copy_examples=CopyExampleGen(input_dict=tfrecords_dict)

context.run(copy_examples)


# """Call StatisticsGen Component"""
# statistics_gen=tfx.components.StatisticsGen(examples=copy_examples.outputs['output_example'])

# context.run(statistics_gen)



INFO:absl:Running driver for CopyExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for CopyExampleGen
INFO:absl:Running publisher for CopyExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CopyExampleGen
    execution_id: 1
    outputs:
        output_example: OutputChannel(artifact_type=Examples, producer_component_id=CopyExampleGen, output_key=output_example, additional_properties={}, additional_custom_properties={}))